In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import xobjects as xo
import xtrack as xt
import xpart as xp
import xcoll as xc

# ============================================
# From line
# ============================================

# Get line and collimators
line = xt.Line.from_json(xc._pkg_root / '..' / 'examples' / 'machines' / 'lhc_run3_b1.json')

coll_manager = xc.CollimatorDatabase.from_yaml(xc._pkg_root / '..' / 'examples' / 'colldb' / 'lhc_run3.yaml', beam=1)
coll_manager.install_everest_collimators(verbose=True, line=line)
df_with_coll = line.check_aperture()
assert not np.any(df_with_coll.has_aperture_problem)

Loading line from dict:   0%|          | 0/101993 [00:00<?, ?it/s]

Done loading line from dict.           
Installing tdisb.a4l2.b1        as EverestCollimator
Installing tcdqa.b4r6.b1        as EverestCollimator
Installing tcsg.b4l7.b1         as EverestCollimator
Installing tcsg.6r7.b1          as EverestCollimator
Installing tcsg.d5r7.b1         as EverestCollimator
Installing tcp.c6l7.b1          as EverestCollimator
Installing tctph.4l1.b1         as EverestCollimator
Installing tcla.b5r3.b1         as EverestCollimator
Installing tcsg.a6l7.b1         as EverestCollimator
Installing tcsg.e5r7.b1         as EverestCollimator
Installing tcld.a11r2.b1        as EverestCollimator
Installing tcsg.a4r7.b1         as EverestCollimator
Installing tcspm.b4l7.b1        as EverestCollimator
Installing tcla.a7r7.b1         as EverestCollimator
Installing tcsg.a5r3.b1         as EverestCollimator
Installing tcsp.a4r6.b1         as EverestCollimator
Installing tctph.4l2.b1         as EverestCollimator
Installing tcla.a6r7.b1         as EverestCollimator
Instal

Slicing line:   0%|          | 0/101993 [00:00<?, ?it/s]

Checking aperture:   0%|          | 0/101944 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
0 thick elements miss associated aperture (upstream or downstream):
[]


In [2]:

# ============================================
# With crystal
# ============================================
coll_cry = xc.EverestCrystal(length=0.002, material=xc.materials.SiliconCrystal, bending_angle=149e-6,
                         width=0.002, height=0.05, side='+', lattice='strip', jaw=0.001)

num_part = int(50000)
x_init   = np.random.normal(loc=1.5e-3, scale=75.e-6, size=num_part)
px_init  = np.random.uniform(low=-50.e-6, high=250.e-6, size=num_part)
y_init   = np.random.normal(loc=0., scale=1e-3, size=num_part)
py_init  = np.random.normal(loc=0., scale=5.e-6, size=num_part)
part = xp.Particles(x=x_init, px=px_init, y=y_init, py=py_init, delta=0, p0c=4e11)

impacts_crystal = xc.InteractionRecord.start(elements=[coll_cry], names='TPCH')
coll_cry.track(part)
part.sort(interleave_lost_particles=True)

impacts_crystal.to_pandas()
df_crystal = impacts_crystal.interactions_per_collimator()


In [6]:
df_crystal

pid    turn
0      0                [MCS]
1      0                [MCS]
2      0       [MCS, VR, MCS]
3      0               [AMVR]
4      0               [AMVR]
                    ...      
49995  0       [MCS, VR, MCS]
49996  0       [MCS, VR, MCS]
49997  0       [MCS, VR, MCS]
49998  0                [MCS]
49999  0       [MCS, VR, MCS]
Name: int, Length: 50000, dtype: object

In [ ]:
def int_list(ids, df=df):
    result = df[ids]
    r2 = np.unique(result, return_counts=True)
    print('Total number: ', r2[1].sum())
    for inter, n in zip(r2[0], r2[1]):
        print(inter, n)